# Final Project - Chicago Business Intelligence for Strategic Planning

In this project, you have been tasked as a full-stack developer to build an application that will be used by data scientists and business analysts for exploratory data analysis and to create different business intelligence reports for the city of Chicago; these reports will be utilized in the strategic planning and the industrial and neighborhood infrastructure investments. The City of Chicago publishes and updates its datasets on its data portal server (https://data.cityofchicago.org/ ) in 16 categories. The 3 categories that this project will utilize for exploratory data analysis and creating the business intelligence reports are: Transportation, Buildings, and Health & Human Services.

## Connect to Postgres

Before running any cells, ensure that the virtual environment is activated by navigating to the `MSDS-432` directory and executing `.\venv\Scripts\activate`. Additionally, ensure all dependencies from the `requirements.txt` file are installed in the virtual environment.

In [ ]:
import dotenv
import numpy as np
import os
import pandas as pd
import psycopg2
from sklearn.linear_model import LinearRegression
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)

In [3]:
# Load environment variables
dotenv.load_dotenv()

DB_NAME = os.getenv("POSTGRES_DB")
DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_HOST = os.getenv("POSTGRES_HOST")
DB_PORT = os.getenv("POSTGRES_PORT")

try:
    # Establish connection
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print("Connected to PostgreSQL successfully!")

except Exception as e:
    print("Error connecting to PostgreSQL:", e)


Connected to PostgreSQL successfully!


## Requirement 1

The business intelligence reports are geared toward tracking and forecasting events that have direct or indirect negative or positive impacts on businesses and neighborhoods in different zip codes within the city of Chicago. The business intelligence reports will be used to send alerts to taxi drivers about the state of COVID-19 in the different zip codes in order to avoid taxi drivers to be the super spreaders in the different zip codes and neighborhoods. For this report, the taxi trips and daily COVID19 datasets for the city of Chicago will be used.

In [7]:
r1a_cur = conn.cursor()

r1a_cur.execute("""
                SELECT 
                    c.week_start, 
                    c.week_end, 
                    c.zip_code, 
                    SUM(c.cases_cumulative) AS total_cases, 
                    t.pickup_zipcode, 
                    t.dropoff_zipcode, 
                    t.trip_start_timestamp
                FROM 
                    public.covid_cases c
                JOIN 
                    taxi_trips t
                    ON t.trip_start_timestamp BETWEEN c.week_start AND c.week_end
                    AND (t.pickup_zipcode = c.zip_code OR t.dropoff_zipcode = c.zip_code)
                GROUP BY 
                    c.week_start, c.week_end, c.zip_code, t.pickup_zipcode, t.dropoff_zipcode, t.trip_start_timestamp
                ORDER BY 
                    total_cases DESC;
               """
)

covid_state = r1a_cur.fetchall()

r1a_cur.close()

covid_state_df = pd.DataFrame(covid_state, columns=["Week Start", "Week End", "Zip Code", "Total Cases", "Pickup Zipcode", "Dropoff Zipcode", "Trip Start"])

covid_state_df

,Week Start,Week End,Zip Code,Total Cases,Pickup Zipcode,Dropoff Zipcode,Trip Start
0,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,None,60625,2023-12-31T23:30:00Z
1,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60638,60625,2023-12-31T23:00:00Z
2,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60625,60646,2023-12-31T23:45:00Z
3,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60610,60625,2023-12-31T23:15:00Z
4,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,None,60625,2023-12-31T22:30:00Z
5,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60640,60625,2023-12-31T23:30:00Z
6,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60625,60660,2023-12-31T23:30:00Z
7,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60625,23299.0,60626,60625,2023-12-31T23:45:00Z
8,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60626,14474.0,60657,60626,2023-12-31T23:30:00Z
9,2023-12-31T00:00:00Z,2024-01-06T00:00:00Z,60626,14474.0,60626,60610,2023-12-31T23:15:00Z


The City of Chicago is also interested to forecast COVID-19 alerts (Low, Medium, High) on daily/weekly basis to the residents of the different neighborhoods considering the counts of the taxi trips and COVID-19 positive test cases.

In [ ]:
r1b_cur = conn.cursor()

r1b_cur.execute("""
                SELECT 
                    c.week_start, 
                    c.week_end, 
                    c.zip_code, 
                    SUM(c.cases_cumulative) AS total_cases, 
                    t.pickup_zipcode, 
                    t.dropoff_zipcode, 
                    t.trip_start_timestamp
                FROM 
                    public.covid_cases c
                JOIN 
                    transportation_trips t
                    ON t.trip_start_timestamp BETWEEN c.week_start AND c.week_end
                    AND (t.pickup_zipcode = c.zip_code OR t.dropoff_zipcode = c.zip_code)
                GROUP BY 
                    c.week_start, c.week_end, c.zip_code, t.pickup_zipcode, t.dropoff_zipcode, t.trip_start_timestamp
                ORDER BY 
                    total_cases DESC;
               """
)

covid_alert = r1b_cur.fetchall()
r1b_cur.close()

covid_alert_df = pd.DataFrame(covid_alert, columns=["week_start", "week_end", "zip_code", "total_cases", "pickup_zipcode", "dropoff_zipcode", "trip_start"])

covid_alert_df["week_start"] = pd.to_datetime(covid_alert_df["week_start"])
covid_alert_df.sort_values(by=["zip_code", "week_start"], inplace=True)

# Loop through each unique zip code
zip_codes = covid_alert_df["zip_code"].unique()
forecast_results = []

for zip_code in zip_codes:
    df_zip = covid_alert_df[covid_alert_df["zip_code"] == zip_code].set_index("week_start")

    df_zip = df_zip.resample('W').sum()

    # Check if there's enough data
    if df_zip.empty or df_zip["total_cases"].isna().all():
        print(f"No data available for ZIP code {zip_code}. Skipping.")
        continue

    df_zip['week_num'] = np.arange(len(df_zip))

    X = df_zip[['week_num']]
    y = df_zip['total_cases']

    model = LinearRegression()
    model.fit(X, y)

    last_week_num = df_zip['week_num'].iloc[-1]
    forecast_weeks = np.arange(last_week_num + 1, last_week_num + 5).reshape(-1, 1)
    forecast_cases = model.predict(forecast_weeks)

    forecast_dates = pd.date_range(df_zip.index[-1] + pd.Timedelta(weeks=1), periods=4, freq='W')

    low, high = np.percentile(df_zip["total_cases"].dropna(), [33, 66])

    def categorize_cases(value):
        if value <= low:
            return "Low"
        elif value <= high:
            return "Medium"
        else:
            return "High"

    categories = [categorize_cases(x) for x in forecast_cases]

    forecast_results.append(pd.DataFrame({"Zip Code": zip_code, 
                                          "Forecasted Cases": forecast_cases, 
                                          "Category": categories, 
                                          "Forecast Date": forecast_dates}))

final_forecast = pd.concat(forecast_results)

print(final_forecast)


  Zip Code  Forecasted Cases Category             Forecast Date
0    60608      10342.555556     High 2022-06-12 00:00:00+00:00
1    60608      10971.555556     High 2022-06-19 00:00:00+00:00
2    60608      11600.555556     High 2022-06-26 00:00:00+00:00
3    60608      12229.555556     High 2022-07-03 00:00:00+00:00
0    60610      25308.623077   Medium 2023-01-01 00:00:00+00:00
1    60610      24932.816886   Medium 2023-01-08 00:00:00+00:00
2    60610      24557.010694   Medium 2023-01-15 00:00:00+00:00
3    60610      24181.204503   Medium 2023-01-22 00:00:00+00:00
0    60611       8383.888462     High 2023-01-01 00:00:00+00:00
1    60611       8073.885460     High 2023-01-08 00:00:00+00:00
2    60611       7763.882458     High 2023-01-15 00:00:00+00:00
3    60611       7453.879456     High 2023-01-22 00:00:00+00:00
0    60612       4369.079618     High 2022-07-24 00:00:00+00:00
1    60612       4421.325203     High 2022-07-31 00:00:00+00:00
2    60612       4473.570788     High 20

## Requirement 2

There are two major airports within the city of Chicago: O’Hare and Midway. And the City of Chicago is interested to track trips from these airports to the different zip codes and the reported COVID-19 positive test cases. The city of Chicago is interested to monitor the traffic of the taxi trips from these airports to the different neighborhoods and zip codes.

In [5]:
r2_cur = conn.cursor()

r2_cur.execute("""
                SELECT pickup_zipcode,
                    pickup_community_area,
                    dropoff_zipcode,
                    dropoff_community_area,
                    covid_cases_in_dropoff
                FROM
                    (SELECT pickup_zipcode,
                            pickup_community_area,
                            dropoff_zipcode,
                            dropoff_community_area
                    FROM taxi_trips
                    WHERE pickup_zipcode = '60666'
                        OR pickup_zipcode = '60638'
                        OR pickup_community_area = '76'
                        OR pickup_community_area = '56'
                        OR pickup_community_area = '64'
                    UNION SELECT pickup_zipcode,
                                pickup_community_area,
                                dropoff_zipcode,
                                dropoff_community_area
                    FROM transportation_trips
                    WHERE pickup_zipcode = '60666'
                        OR pickup_zipcode = '60638'
                        OR pickup_community_area = '76'
                        OR pickup_community_area = '56'
                        OR pickup_community_area = '64' ) AS trips
                JOIN
                    (SELECT zip_code,
                            MAX(cases_cumulative) AS covid_cases_in_dropoff
                    FROM covid_cases
                    GROUP BY zip_code) AS covid ON trips.dropoff_zipcode = covid.zip_code;
               """
)

trips_from_airport_covid = r2_cur.fetchall()

r2_cur.close()

trips_from_airport_covid_df = pd.DataFrame(trips_from_airport_covid, columns=["Pickup Zipcode", "Pickup Community", "Dropoff Zipcode", "Dropoff Community", "Covid Cases in Dropoff Area"])

trips_from_airport_covid_df

,Pickup Zipcode,Pickup Community,Dropoff Zipcode,Dropoff Community,Covid Cases in Dropoff Area
0,None,76,60612,28,11270.0
1,None,76,60622,24,15840.0
2,60638,56,60605,32,8042.0
3,None,76,60618,5,26808.0
4,60638,56,60611,8,9921.0
...,...,...,...,...,...
61,None,76,60641,16,23128.0
62,60638,56,60625,14,23630.0
63,60638,56,60610,8,12122.0
64,60638,56,60657,6,20214.0


## Requirement 3

The city of Chicago has created the COVID-19 Community Vulnerability Index (CCVI) to identify communities that have been disproportionately affected by COVID-19 and are vulnerable to barriers to COVID-19 vaccine uptake. The city of Chicago is interested to track the number of taxi trips from/to the neighborhoods that have CCVI Category with value HIGH

In [ ]:
r3_cur = conn.cursor()

r3_cur.execute("""
                SELECT DISTINCT taxi.trip_id,
                                taxi.trip_start_timestamp,
                                taxi.trip_end_timestamp,
                                taxi.pickup_zipcode,
                                taxi.pickup_community_area,
                                taxi.dropoff_zipcode,
                                taxi.dropoff_community_area,
                                ccvi.ccvi_category
                FROM covid_vulnerability_index AS ccvi
                JOIN taxi_trips AS taxi ON ccvi.community_area_or_zip IN ( taxi.pickup_community_area,
                                                                        taxi.pickup_zipcode,
                                                                        taxi.dropoff_community_area,
                                                                        taxi.dropoff_zipcode )
                WHERE ccvi.ccvi_category = 'HIGH';
               """
)

high_ccvi_trips = r3_cur.fetchall()

r3_cur.close()

high_ccvi_trips_df = pd.DataFrame(high_ccvi_trips, columns=["Trip ID", "Trip Start", "Trip End", "Pickup Zipcode", "Pickup Community", "Dropoff Zipcode", "Dropoff Community", "CCVI Category"])

high_ccvi_trips_df

,Trip ID,Trip Start,Trip End,Pickup Zipcode,Pickup Community,Dropoff Zipcode,Dropoff Community,CCVI Category
0,1c94038fa2e65807fb8faa1b4141a0ce1ddb5140,2023-12-31T23:15:00Z,2023-12-31T23:30:00Z,60638,56,60618,5,HIGH
1,d73a7fd4c5582a3b74d45e363017b1bbc7fa3c6f,2023-12-31T23:00:00Z,2023-12-31T23:15:00Z,60638,56,60657,6,HIGH
2,9e91e9b8e3e79aed101faeba293167d136e77bb4,2023-12-31T23:00:00Z,2023-12-31T23:15:00Z,60604,32,60608,31,HIGH
3,0461556275a2f90fef2846c888e307f18d61619f,2023-12-31T23:30:00Z,2024-01-01T00:00:00Z,60609,37,60628,49,HIGH
4,ea9d5340b701df5113e84afc7eb832b770b91174,2023-12-31T23:15:00Z,2023-12-31T23:30:00Z,60610,8,60644,25,HIGH
5,93a1f836379b26f43829d36a7a03411d401ce3a6,2023-12-31T23:00:00Z,2023-12-31T23:15:00Z,60621,68,60628,49,HIGH
6,9937d46dbc7b1eb988caa86f5ecbe0141c199cc5,2023-12-31T23:00:00Z,2023-12-31T23:30:00Z,60638,56,60622,24,HIGH
7,0e0ed752d2feba250d6cf8c9c9f31a55f2b3f3ae,2023-12-31T22:45:00Z,2023-12-31T23:00:00Z,60617,51,60616,35,HIGH
8,a6630e284573892bbafeae2ab3f2de27b18e4988,2023-12-31T23:15:00Z,2023-12-31T23:45:00Z,60638,56,60614,7,HIGH
9,d45e012bbd6fffa5ffa8aba12b6d61961c89e9e0,2023-12-31T23:30:00Z,2024-01-01T00:00:00Z,60638,56,60607,28,HIGH


## Requirement 4

For streetscaping investment and planning, the city of Chicago is interested to forecast daily, weekly, and monthly traffic patterns utilizing the taxi trips for the different zip codes.

In [33]:
r4_cur = conn.cursor()

r4_cur.execute("""
                SELECT trip_start_timestamp, trip_end_timestamp, pickup_zipcode, dropoff_zipcode
                FROM transportation_trips;
               """
)

traffic_pattern = r4_cur.fetchall()
r4_cur.close()

traffic_pattern_df = pd.DataFrame(traffic_pattern, columns=["start_timestamp", "end_timestamp", "pickup_zipcode", "dropoff_zipcode"])

traffic_pattern_df["start_timestamp"] = pd.to_datetime(traffic_pattern_df["start_timestamp"])
traffic_pattern_df["end_timestamp"] = pd.to_datetime(traffic_pattern_df["end_timestamp"])

traffic_pattern_df["date"] = traffic_pattern_df["start_timestamp"].dt.date
traffic_pattern_df["week"] = traffic_pattern_df["start_timestamp"].dt.to_period("W")
traffic_pattern_df["month"] = traffic_pattern_df["start_timestamp"].dt.to_period("M")

daily_traffic = traffic_pattern_df.groupby(["pickup_zipcode", "date"]).size().reset_index(name="trip_count")
weekly_traffic = traffic_pattern_df.groupby(["pickup_zipcode", "week"]).size().reset_index(name="trip_count")
monthly_traffic = traffic_pattern_df.groupby(["pickup_zipcode", "month"]).size().reset_index(name="trip_count")

# Function to predict traffic for the next 4 periods (daily, weekly, or monthly)
def predict_traffic(traffic_data, period_type='daily'):
    zip_codes = traffic_data["pickup_zipcode"].unique()
    forecast_results = []
    
    for zip_code in zip_codes:
        df_zip = traffic_data[traffic_data["pickup_zipcode"] == zip_code]

        if period_type == 'daily':
            df_zip = df_zip.set_index("date")
        elif period_type == 'weekly':
            df_zip = df_zip.set_index("week")
        elif period_type == 'monthly':
            df_zip = df_zip.set_index("month")
        else:
            raise ValueError("Invalid period_type. Choose from 'daily', 'weekly', or 'monthly'.")

        df_zip = df_zip.sort_index()

        if df_zip.empty or df_zip["trip_count"].isna().all():
            print(f"No data available for ZIP code {zip_code}. Skipping.")
            continue

        df_zip['period_num'] = np.arange(len(df_zip))

        X = df_zip[['period_num']]
        y = df_zip['trip_count']

        model = LinearRegression()
        model.fit(X, y)

        # Forecast the next 4 periods
        last_period_num = df_zip['period_num'].iloc[-1]
        forecast_periods = np.arange(last_period_num + 1, last_period_num + 5).reshape(-1, 1)
        forecast_traffic = model.predict(forecast_periods)

        if period_type == 'daily':
            forecast_dates = pd.date_range(df_zip.index[-1] + pd.Timedelta(days=1), periods=4, freq='D')
        elif period_type == 'weekly':
            forecast_dates = pd.date_range(df_zip.index[-1].start_time + pd.Timedelta(weeks=1), periods=4, freq='W')
        else:
            forecast_dates = pd.date_range(df_zip.index[-1].start_time + pd.Timedelta(days=30), periods=4, freq='M')

        forecast_results.append(pd.DataFrame({"Zip Code": zip_code, 
                                              "Forecasted Traffic": forecast_traffic, 
                                              "Forecast Date": forecast_dates}))

    final_forecast = pd.concat(forecast_results)

    return final_forecast

forecasted_daily_traffic = predict_traffic(daily_traffic, period_type='daily')
forecasted_weekly_traffic = predict_traffic(weekly_traffic, period_type='weekly')
forecasted_monthly_traffic = predict_traffic(monthly_traffic, period_type='monthly')

print("forecasted_daily_traffic\n", forecasted_daily_traffic)
print("forecasted_weekly_traffic\n", forecasted_weekly_traffic)
print("forecasted_monthly_traffic\n", forecasted_monthly_traffic)


forecasted_daily_traffic
    Zip Code  Forecasted Traffic Forecast Date
0     60601            1.077996    2022-12-31
1     60601            1.074137    2023-01-01
2     60601            1.070278    2023-01-02
3     60601            1.066419    2023-01-03
0     60603            1.446747    2022-12-30
..      ...                 ...           ...
3     60661            1.030391    2023-01-03
0     60707            1.000000    2022-05-26
1     60707            1.000000    2022-05-27
2     60707            1.000000    2022-05-28
3     60707            1.000000    2022-05-29

[212 rows x 3 columns]
forecasted_weekly_traffic
    Zip Code  Forecasted Traffic Forecast Date
0     60601            2.628232    2023-01-08
1     60601            2.607306    2023-01-15
2     60601            2.586380    2023-01-22
3     60601            2.565453    2023-01-29
0     60603            4.591837    2023-01-08
..      ...                 ...           ...
3     60661            2.164973    2023-01-29
0  

## Requirement 5

For industrial and neighborhood infrastructure investment, the city of Chicago is interested to invest in top 5 neighborhoods with highest unemployment rate and poverty rate and waive the fees for building permits in those neighborhoods in order to encourage businesses to develop and invest in those neighborhoods. Both, building permits and unemployment, datasets will be used in this report.

In [5]:
r5_cur = conn.cursor()

r5_cur.execute("""
                SELECT bp.id,
                    bp.community_area,
                    bp.permit_type,
                    bp.total_fee,
                    bp.reported_cost
                FROM building_permits AS bp
                JOIN
                    (SELECT community_area_number,
                            percent_aged_16_unemployed,
                            per_capita_income
                    FROM census_data
                    ORDER BY percent_aged_16_unemployed,
                            per_capita_income DESC
                    LIMIT 5) AS cd ON bp.community_area = cd.community_area_number;
               """
)

waive_permit_fees = r5_cur.fetchall()

r5_cur.close()

waive_permit_fees_df = pd.DataFrame(waive_permit_fees, columns=["Permit ID", "Community Area", "Permit Type", "Total Fee", "Reported Cost"])

waive_permit_fees_df

,Permit ID,Community Area,Permit Type,Total Fee,Reported Cost
0,3185457,6,PERMIT - NEW CONSTRUCTION,2575.0,750000
1,1972144,32,PERMIT – EXPRESS PERMIT PROGRAM,50.0,51540
2,3206816,6,PERMIT - RENOVATION/ALTERATION,2299.0,24000
3,3236235,7,PERMIT - RENOVATION/ALTERATION,2803.1,1596000
4,3240342,7,PERMIT - RENOVATION/ALTERATION,828.9,245000
...,...,...,...,...,...
127,2135372,32,PERMIT – EXPRESS PERMIT PROGRAM,50.0,29800
128,2134971,32,PERMIT – EXPRESS PERMIT PROGRAM,50.0,9400
129,2157780,32,PERMIT – EXPRESS PERMIT PROGRAM,50.0,10668
130,2157476,33,PERMIT – EXPRESS PERMIT PROGRAM,50.0,5000


## Requirement 6

According to a report published by Crain’s Chicago Business, The “little guys”, small businesses, have trouble competing with the big players like Amazon and Walmart for warehouse spaces. To help small business, assume a new imaginary program has been piloted with the name Illinois Small Business Emergency Loan Fund Delta to offer small businesses low interest loans of up to $250,000 for those applicants with PERMIT_TYPE of PERMIT - NEW CONSTRUCTION in the zip code that has the lowest number of PERMIT - NEW CONSTRUCTION applications and PER CAPITA INCOME is less than 30,000 for the planned construction site. Both, building permits and unemployment, datasets will be used in this report.

In [6]:
r6_cur = conn.cursor()

r6_cur.execute("""
                SELECT building_permits.id,
                    building_permits.community_area,
                    building_permits.zipcode,
                    building_permits.permit_type,
                    building_permits.total_fee,
                    building_permits.reported_cost
                FROM building_permits
                JOIN
                    (SELECT permit_type,
                            zipcode,
                            COUNT(*) AS num_permits
                        FROM building_permits
                        WHERE permit_type = 'PERMIT - NEW CONSTRUCTION'
                        GROUP BY permit_type,
                                zipcode
                        ORDER BY num_permits ASC) AS bp ON building_permits.zipcode = bp.zipcode
                JOIN
                    (SELECT community_area_number
                        FROM census_data
                        WHERE per_capita_income < 30000) AS cd ON building_permits.community_area = cd.community_area_number WHERE building_permits.permit_type = 'PERMIT - NEW CONSTRUCTION'
                AND CAST(reported_cost AS INT) <= 250000;
               """
)

small_business_loans = r6_cur.fetchall()

r6_cur.close()

small_business_loans_df = pd.DataFrame(small_business_loans, columns=["Permit ID", "Community Area", "Zipcode", "Permit Type", "Total Fee", "Reported Cost"])

small_business_loans_df

,Permit ID,Community Area,Zipcode,Permit Type,Total Fee,Reported Cost
0,3217660,42,60637,PERMIT - NEW CONSTRUCTION,525.0,60000
1,1928003,42,60637,PERMIT - NEW CONSTRUCTION,1873.7,100000
2,3265607,43,60619,PERMIT - NEW CONSTRUCTION,4128.0,160000
3,3383361,25,60707,PERMIT - NEW CONSTRUCTION,377.0,1
4,3384285,73,60628,PERMIT - NEW CONSTRUCTION,377.0,150000


In [ ]:
# Close the Postgres connection
conn.close()